In [2]:
!gdown https://drive.google.com/uc?id=1pPaY2P-xrNVwx3PXu9fyncmxFjlrD1uZ

Downloading...
From (original): https://drive.google.com/uc?id=1pPaY2P-xrNVwx3PXu9fyncmxFjlrD1uZ
From (redirected): https://drive.google.com/uc?id=1pPaY2P-xrNVwx3PXu9fyncmxFjlrD1uZ&confirm=t&uuid=b78a9159-30dc-4e2a-b5fd-c7711adc4b97
To: /content/to_download.tar.gz
100% 2.87G/2.87G [00:47<00:00, 59.8MB/s]


### Extract

In [3]:
%%bash
tar -xzf to_download.tar.gz

In [4]:
# Hücre 2
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Extra'yı train'e alma

In [5]:
# dosyaları bulunduğu yerden doğrudan çalıştır
%run /content/drive/MyDrive/Thermal-Image-Processing/move_extra_to_train.py --processed_dir ./processed --start_idx 70000

Done — moved 2706 extra image/label pairs into train starting at 70000


### Extradan gelen 70k'lardakilerin class problemini çözme

In [6]:
%run /content/drive/MyDrive/Thermal-Image-Processing/fix_class_in_70ks.py --labels-dir ./processed/train/labels --start-id 70000 --input-class 2 --output-class 0

Processed 70000.txt: 1 lines kept
Processed 70001.txt: 1 lines kept
Processed 70002.txt: 1 lines kept
Processed 70003.txt: 1 lines kept
Processed 70004.txt: 1 lines kept
Processed 70005.txt: 1 lines kept
Processed 70006.txt: 1 lines kept
Processed 70007.txt: 1 lines kept
Processed 70008.txt: 1 lines kept
Processed 70009.txt: 1 lines kept
Processed 70010.txt: 1 lines kept
Processed 70011.txt: 1 lines kept
Processed 70012.txt: 1 lines kept
Processed 70013.txt: 1 lines kept
Processed 70014.txt: 1 lines kept
Processed 70015.txt: 1 lines kept
Processed 70016.txt: 1 lines kept
Processed 70017.txt: 2 lines kept
Processed 70018.txt: 2 lines kept
Processed 70019.txt: 2 lines kept
Processed 70020.txt: 1 lines kept
Processed 70021.txt: 1 lines kept
Processed 70022.txt: 1 lines kept
Processed 70023.txt: 1 lines kept
Processed 70024.txt: 1 lines kept
Processed 70025.txt: 1 lines kept
Processed 70026.txt: 1 lines kept
Processed 70027.txt: 1 lines kept
Processed 70028.txt: 1 lines kept
Processed 7002

##### Train'deki 70k sonrasının %10'unu val'a , kalan %10'ununu test'e çekme

In [7]:
%run /content/drive/MyDrive/Thermal-Image-Processing/split_70ks_to_val_and_test.py --processed_dir ./processed --start_idx 70000 --val_ratio 0.1 --test_ratio 0.1

Moving 271 -> val (indices 72435–72705)
Moving 244 -> test (indices 72191–72434)
Done.


# Kurulum

In [8]:
# %%
# Cell 1: Kurulum
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

Import ve fonksiyonlar

In [9]:
# Cell 2: Tanımlar ve Fonksiyonlar
import sys
from pathlib import Path
from ultralytics import YOLO

# Eğitim sırasında hariç tutulacak ID aralıkları (1920×480, PNG frameler)
EXCLUDE_RANGES = [(8000, 9999), (12000, 13999)]
# Geçerli resim uzantıları
VALID_EXTS = {".jpg", ".jpeg", ".png"}

def is_excluded(idx: int) -> bool:
    return any(start <= idx <= end for start, end in EXCLUDE_RANGES)

def sanity_check_and_clean(data_root: Path):
    for split in ("train", "val", "test"):
        img_dir = data_root / split / "images"
        lbl_dir = data_root / split / "labels"
        if not img_dir.is_dir() or not lbl_dir.is_dir():
            raise FileNotFoundError(f"Eksik klasör: {img_dir} veya {lbl_dir}")
        if split == "train":
            for img_path in img_dir.iterdir():
                if img_path.suffix.lower() not in VALID_EXTS:
                    continue
                stem = img_path.stem
                if stem.isdigit() and is_excluded(int(stem)):
                    img_path.unlink()
                    lbl = lbl_dir / f"{stem}.txt"
                    if lbl.exists():
                        lbl.unlink()
        imgs = {p.stem for p in img_dir.iterdir() if p.suffix.lower() in VALID_EXTS}
        lbls = {p.stem for p in lbl_dir.glob("*.txt")}
        missing_lbl = imgs - lbls
        missing_img = lbls - imgs
        if missing_lbl or missing_img:
            raise ValueError(f"{split}: Eksik etiket: {missing_lbl}, Eksik görüntü: {missing_img}")

def write_data_yaml(data_root: Path, out_path="data.yaml"):
    content = f"""
path: {data_root}
train: train/images
val:   val/images

test:  test/images

names:
  0: person
"""
    with open(out_path, "w") as f:
        f.write(content.strip())
    print(f"[OK] data.yaml oluşturuldu: {out_path}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Eğitim hiperparametreleri

In [10]:
# %%
# Cell 3: Argümanları Manuel Tanımla (Notebook için)
data_root       = "/content/processed"   # Colab üzerinde veriniz altındaki train/val/test klasörleri
model_weights   = "yolov11n.pt"
epochs          = 100
batch           = 16
imgsz           = 640
device          = "0"

Temizlik & YAML hazırlığı

In [11]:
# %%
# Cell 4: Temizlik & YAML hazırlığı
from pathlib import Path
import sys

# EXCLUDE_RANGES ve VALID_EXTS doğrudan buraya kopyala:
EXCLUDE_RANGES = [(8000, 9999), (12000, 13999)]
VALID_EXTS = {".jpg", ".jpeg", ".png"}

def is_excluded(idx: int) -> bool:
    return any(start <= idx <= end for start, end in EXCLUDE_RANGES)

def sanity_check_and_clean(data_root: Path):
    for split in ("train", "val", "test"):
        img_dir = data_root / split / "images"
        lbl_dir = data_root / split / "labels"
        if not img_dir.is_dir() or not lbl_dir.is_dir():
            print(f"[ERROR] Eksik klasör: {img_dir} veya {lbl_dir}", file=sys.stderr)
            sys.exit(1)
        if split == "train":
            for img_path in img_dir.iterdir():
                if img_path.suffix.lower() not in VALID_EXTS: continue
                if img_path.stem.isdigit() and is_excluded(int(img_path.stem)):
                    img_path.unlink()
                    (lbl_dir / f"{img_path.stem}.txt").unlink(missing_ok=True)
        imgs = {p.stem for p in img_dir.iterdir() if p.suffix.lower() in VALID_EXTS}
        lbls = {p.stem for p in lbl_dir.glob("*.txt")}
        if diff := imgs - lbls:
            print(f"[ERROR] {split}: Etiketi eksik: {list(diff)[:5]}", file=sys.stderr)
            sys.exit(1)
        if diff := lbls - imgs:
            print(f"[ERROR] {split}: Görüntüsü eksik: {list(diff)[:5]}", file=sys.stderr)
            sys.exit(1)
    print("[OK] Sanity-check ve temizleme tamam.")

def write_data_yaml(data_root: Path, out_path="data.yaml"):
    content = f"""
path: {data_root}
train: train/images
val:   val/images
test:  test/images

names:
  0: person
"""
    (Path.cwd()/out_path).write_text(content.strip())
    print(f"[OK] data.yaml oluşturuldu: {out_path}")

# Çalıştır:
root = Path(data_root)
sanity_check_and_clean(root)
write_data_yaml(root)


[OK] Sanity-check ve temizleme tamam.
[OK] data.yaml oluşturuldu: data.yaml


* Yolo base modelini drive'dan çekme (nano)

In [12]:
!cp /content/drive/MyDrive/Thermal-Image-Processing/yolov11n.pt /content/yolov11n.pt

## TRAIN

In [ ]:
# %%
# Cell 5: Eğitim
from ultralytics import YOLO

model = YOLO(model_weights)
model.train(
    data="data.yaml",
    epochs=epochs,
    batch=batch,
    imgsz=imgsz,
    device=device,
    multi_scale=True,
    patience=10,
    augment=True,
    lr0=0.02,
    momentum=0.937,
    weight_decay=0.0005,
)


Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.02, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov11n.pt, momentum=0.937, mosaic=1.0, multi_scale=True, name=train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, p

100%|██████████| 755k/755k [00:00<00:00, 122MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

 10                  -1  1    249728  ultralytics.nn.modules.block.C2PSA           [256, 256, 1]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  1    111296  ultralytics.nn.modules.block.C3k2            [384, 128, 1, False]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1     32096  ultralytics.nn.modules.block.C3k2            [256, 64, 1, False]           
 17                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
 18            [-1, 13]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

100%|██████████| 5.35M/5.35M [00:00<00:00, 377MB/s]


...

                   all       2197       5951      0.956       0.95      0.981      0.718



100 epochs completed in 3.561 hours.
Optimizer stripped from runs/detect/train11/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train11/weights/best.pt, 5.5MB

Validating runs/detect/train11/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 69/69 [00:12<00:00,  5.51it/s]


                   all       2197       5951      0.946      0.952       0.98      0.707
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train11


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78f6900bf710>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# Validation & Test

In [ ]:
# %%
# Cell 6: Validation & Test Değerlendirmesi
print("\n[INFO] Validation seti üzerinde değerlendirme:")
best_pt = "/content/runs/detect/train/weights/best.pt"
model = YOLO(best_pt)
# %%
# Cell X: Extract ALL metrics including F1
results = model.val(data="data.yaml", batch=batch, imgsz=imgsz, conf=0.001)

# 1) Print whatever metrics the object has in its results_dict
print("\n[ALL AVAILABLE METRICS]")
for name, val in results.results_dict.items():
    print(f"{name:20s}: {val:.4f}")

# 2) Compute F1 = 2·P·R / (P+R)
P = results.results_dict['metrics/precision(B)']
R = results.results_dict['metrics/recall(B)']
F1 = 2 * P * R / (P + R + 1e-16)
print(f"\nComputed F1 Score       : {F1:.4f}")


[INFO] Validation seti üzerinde değerlendirme:
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1649.7±824.1 MB/s, size: 49.1 KB)


val: Scanning /content/processed/val/labels.cache... 2197 images, 23 backgrounds, 0 corrupt: 100%|██████████| 2197/2197 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 138/138 [00:10<00:00, 13.02it/s]


                   all       2197       5951      0.956      0.951       0.98      0.709
                person       2174       5951      0.956      0.951       0.98      0.709
Speed: 0.2ms preprocess, 0.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val21

[ALL AVAILABLE METRICS]
metrics/precision(B): 0.9561
metrics/recall(B)   : 0.9514
metrics/mAP50(B)    : 0.9803
metrics/mAP50-95(B) : 0.7094
fitness             : 0.7365

Computed F1 Score       : 0.9538


# Yalnızca Validation -> Conf Hiperparametre Eğitimi

In [ ]:
# %%
# Cell X: Conf‐Sweep by mAP@0.50-0.95 on VAL set

import numpy as np
import pandas as pd

best_conf, best_map = 0.0, 0.0
records = []

# Sweep conf from 0.20 to 0.30
for c in np.arange(0.20, 0.51, 0.02):
    res = model.val(
        data="data.yaml",
        split="val",
        batch=batch,
        imgsz=imgsz,
        conf=float(c),
        verbose=False
    )
    # Extract metrics
    P   = float(res.results_dict['metrics/precision(B)'])
    R   = float(res.results_dict['metrics/recall(B)'])
    F1  = 2 * P * R / (P + R + 1e-6)
    mAP = float(res.results_dict['metrics/mAP50-95(B)'])
    records.append((c, P, R, F1, mAP))
    if mAP > best_map:
        best_conf, best_map = c, mAP

# Show results
df = pd.DataFrame(records, columns=['conf','precision','recall','F1','mAP50-95'])
display(df)

print(f"\n🔥 Best VAL conf by mAP50-95 = {best_conf:.2f}, mAP50-95 = {best_map:.4f}")


Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1433.0±492.7 MB/s, size: 78.1 KB)


val: Scanning /content/processed/val/labels.cache... 2197 images, 23 backgrounds, 0 corrupt: 100%|██████████| 2197/2197 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 138/138 [00:10<00:00, 13.28it/s]


                   all       2197       5951      0.956      0.951      0.977      0.737
Speed: 0.2ms preprocess, 0.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val55
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1683.8±1296.3 MB/s, size: 86.1 KB)


val: Scanning /content/processed/val/labels.cache... 2197 images, 23 backgrounds, 0 corrupt: 100%|██████████| 2197/2197 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 138/138 [00:10<00:00, 13.28it/s]


                   all       2197       5951      0.956      0.951      0.977      0.737
Speed: 0.2ms preprocess, 0.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val56
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1308.7±987.9 MB/s, size: 178.3 KB)


...

                   all       2197       5951      0.968      0.934       0.96      0.734
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val69
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1847.7±896.2 MB/s, size: 83.9 KB)


val: Scanning /content/processed/val/labels.cache... 2197 images, 23 backgrounds, 0 corrupt: 100%|██████████| 2197/2197 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 138/138 [00:10<00:00, 13.00it/s]


                   all       2197       5951      0.972      0.929      0.957      0.734
Speed: 0.2ms preprocess, 0.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val70


,conf,precision,recall,F1,mAP50-95
0,0.20,0.956104,0.951437,0.953764,0.737208
1,0.22,0.956104,0.951437,0.953764,0.737297
2,0.24,0.956104,0.951437,0.953764,0.737461
3,0.26,0.956104,0.951437,0.953764,0.737270
4,0.28,0.956104,0.951437,0.953764,0.737503
5,0.30,0.956104,0.951437,0.953764,0.737467
6,0.32,0.956104,0.951437,0.953764,0.737322
7,0.34,0.956104,0.951437,0.953764,0.737221
8,0.36,0.954517,0.952159,0.953336,0.737020
9,0.38,0.953323,0.953621,0.953471,0.736808



🔥 Best VAL conf by mAP50-95 = 0.28, mAP50-95 = 0.7375


# Yalnızca Test

In [16]:
# Cell X: Test Seti Değerlendirmesi
print("\n[INFO] Test seti üzerinde değerlendirme:")
!cp /content/drive/MyDrive/Thermal-Image-Processing/runs/best.pt /content/best.pt
model = YOLO("best.pt")
test_results = model.val(
    data="data.yaml",
    split="test",    # <<< burada liste değil, string
    batch=batch,
    imgsz=imgsz,
    conf=0.28
)

# Geri dönen metrics sözlüğünden direkt çekip yazdıralım:
r = test_results.results_dict
print(f"\nPrecision : {r['metrics/precision(B)']:.6f}")
print(f"Recall    : {r['metrics/recall(B)']:.6f}")
print(f"mAP@0.50  : {r['metrics/mAP50(B)']:.6f}")
print(f"mAP@0.50-0.95: {r['metrics/mAP50-95(B)']:.6f}")

# F1 skorunu istersen şöyle hesaplayabilirsin:
prec = r['metrics/precision(B)']
rec  = r['metrics/recall(B)']
f1   = 2 * prec * rec / (prec + rec + 1e-6)
print(f"F1 score : {f1:.6f}")



[INFO] Test seti üzerinde değerlendirme:
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1898.6±507.6 MB/s, size: 75.1 KB)


val: Scanning /content/processed/test/labels.cache... 4898 images, 762 backgrounds, 0 corrupt: 100%|██████████| 4898/4898 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 307/307 [00:19<00:00, 15.60it/s]


                   all       4898      11341      0.939      0.855      0.923      0.652
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val2

Precision : 0.938860
Recall    : 0.854510
mAP@0.50  : 0.922531
mAP@0.50-0.95: 0.652227
F1 score : 0.894701


In [ ]:
import shutil

source_folder = '/content/'
destination_folder = '/content/drive/MyDrive/Thermal-Image-Processing/runs' # Burayı istediğiniz Drive klasör yolu ile değiştirin

try:
    shutil.copytree(source_folder, destination_folder)
    print(f"'{source_folder}' klasörü '{destination_folder}' konumuna başarıyla kopyalandı.")
except FileExistsError:
    print(f"'{destination_folder}' klasörü zaten mevcut. Kopyalama yapılmadı.")
except Exception as e:
    print(f"Hata oluştu: {e}")

'/content/runs' klasörü '/content/drive/MyDrive/Thermal-Image-Processing/runs' konumuna başarıyla kopyalandı.
